In [1]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings("ignore")

import os
os.chdir("/Users/admin/Downloads/EBAC/dataframesEbac")
df = pd.read_csv("kc house data.csv")
df.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900.0,3,1.00,1180,5650,1.0,0,0,...,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,6414100192,20141209T000000,538000.0,3,2.25,2570,7242,2.0,0,0,...,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,5631500400,20150225T000000,180000.0,2,1.00,770,10000,1.0,0,0,...,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
3,2487200875,20141209T000000,604000.0,4,3.00,1960,5000,1.0,0,0,...,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
4,1954400510,20150218T000000,510000.0,3,2.00,1680,8080,1.0,0,0,...,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503


In [2]:
df = df[['bedrooms', 'bathrooms', 'sqft_living',
         'sqft_lot', 'floors', 'waterfront', 'view', 'condition',
         'grade', 'sqft_above', 'sqft_basement', 'lat', 
         'sqft_living15', 'sqft_lot15', "price"]]

In [3]:
from sklearn.preprocessing import RobustScaler
scaler = RobustScaler()
df_new = pd.DataFrame(scaler.fit_transform(df), columns = df.columns)


In [4]:
df_new["Intercepto"] = 1
Xdata = df_new[['Intercepto', 'bedrooms', 'bathrooms', 'sqft_living',
         'sqft_lot', 'floors', 'waterfront', 'view', 'condition',
         'grade', 'sqft_above', 'sqft_basement', 'lat', 
         'sqft_living15', 'sqft_lot15']].values
Ydata = df_new[["price"]].values

In [5]:
# Bases de entrenamiento y de prueba
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(Xdata, Ydata, test_size = 0.3, random_state = 42) 

In [6]:
x = x_train
y = y_train

In [7]:
# Para el formato numerico
np.set_printoptions(formatter = {'float_kind':'{:f}'.format})

In [8]:
XT_X = np.matmul(np.matrix.transpose(x), x)
XT_X_inv = np.linalg.inv(XT_X)
XT_Y = np.matmul(np.matrix.transpose(x), y)
betas = np.matmul(XT_X_inv, XT_Y)
betas

array([[8.180638],
       [0.254874],
       [-0.989000],
       [28.385896],
       [-0.005156],
       [-0.394798],
       [2.090004],
       [0.296333],
       [0.222843],
       [0.128134],
       [-23.990230],
       [-14.110399],
       [0.425847],
       [0.007415],
       [-0.007904]])

In [9]:
# Calculo de los pronosticos para Y de acuerdo a los  coeficientes de regresion
Y_pred = np.matmul(x, betas)
# Calculo de residuales
Resid = y - Y_pred

In [10]:
# Calculo de la suma de residuales al cuadrado
RSS = float(np.matmul(np.matrix.transpose(Resid), Resid))
# Calculo de la suma total de cuadrados
TSS = float( np.matmul(np.matrix.transpose(y), y) - len(y) * (y.mean()**2))
# Calculo del coeficiente de determinación 
R_cuad = float(1 - RSS/TSS)
R_cuad

0.019631305731591442

In [11]:
# Calculo del coeficiente de determinación ajustado
RSqAj = float(1 - (RSS / (x.shape[0] - x.shape[1])) / (TSS / (x.shape[0] - 1)))
RSqAj

0.018723196579827794

In [12]:
# Calculo de la varianza del error de regresion
s_cuad = RSS / (len(y) - x.shape[1])
print('Varianza:', s_cuad)
# Desviacion estandar del error de regresion
import math
s = math.sqrt(s_cuad)
print('Desviacion estandar:', s)

Varianza: 1.228503977515254
Desviacion estandar: 1.1083789864099978


In [13]:
# Calculo de las t's estadisticas para cada coeficiente de regresion
result_t = []
for i in range(0, x.shape[1]):
    t = float(betas[i] / (s * math.sqrt(XT_X_inv[i][i])))
    result_t.append(t)
result_t

[3.039870142583276e-05,
 20.68339753012947,
 -65.02305764273379,
 3.287446881867538e-05,
 -2.7613456027800174,
 -17.201912205291286,
 17.857323091231684,
 21.443221801349523,
 15.285769356698593,
 9.328794966055694,
 -3.058934178098722e-05,
 -3.2770810148563916e-05,
 30.67735959710716,
 0.38706717192240564,
 -3.306093151899892]

# Criterio 1

In [15]:
# Obtener el valor critico de la t de Student de tablas
import scipy.stats

grados_libertad = len(y) - x.shape[1]
# La t critica se obtendra a un nivel de confianza del 95% (Alfa = 5%)
t_critico = abs(scipy.stats.t.ppf(q = 0.025, df = grados_libertad))
t_critico

1.9601209554272305

In [16]:
for i in range(0, x.shape[1]):
    if(abs(result_t[i]) > t_critico):
        print("Beta", i, "es significativa") # Aqui se rechaza H0
    else:
        print("Beta", i, "No significativa") # Aqui NO se rechaza H0

Beta 0 No significativa
Beta 1 es significativa
Beta 2 es significativa
Beta 3 No significativa
Beta 4 es significativa
Beta 5 es significativa
Beta 6 es significativa
Beta 7 es significativa
Beta 8 es significativa
Beta 9 es significativa
Beta 10 No significativa
Beta 11 No significativa
Beta 12 es significativa
Beta 13 No significativa
Beta 14 es significativa


# Criterio 2

In [18]:
# Calculo de los valores p
for i in range(0, x.shape[1]):
    print("valor p de Beta", i, ":", scipy.stats.t.sf(abs(result_t[i]), df = grados_libertad) * 2)

valor p de Beta 0 : 0.9999757457466594
valor p de Beta 1 : 9.672969153123015e-94
valor p de Beta 2 : 0.0
valor p de Beta 3 : 0.9999737704027558
valor p de Beta 4 : 0.00576331893457272
valor p de Beta 5 : 1.0836703989149571e-65
valor p de Beta 6 : 1.3455177456743749e-70
valor p de Beta 7 : 1.6521644238339474e-100
valor p de Beta 8 : 2.3434021526601324e-52
valor p de Beta 9 : 1.2166123287403357e-20
valor p de Beta 10 : 0.9999755936401795
valor p de Beta 11 : 0.9999738531090403
valor p de Beta 12 : 1.514950590598981e-200
valor p de Beta 13 : 0.6987119298279014
valor p de Beta 14 : 0.000948270723247434


# Criterio 3

In [20]:
# Calculo de intevalos de confianza del 95% para el verdadero valor del coeficiente de cada Beta
for i in range (0, x.shape[1]):
    print('El valor de Beta', i, 'se encuentra entre', float(betas[i]) - t_critico * s * math.sqrt(XT_X_inv[i][i]),
         "y", float(betas[i]) + t_critico * s * math.sqrt(XT_X_inv[i][i]))

El valor de Beta 0 se encuentra entre -527482.77514316 y 527499.136419259
El valor de Beta 1 se encuentra entre 0.2307197368393743 y 0.27902737030354596
El valor de Beta 2 se encuentra entre -1.018812992127849 y -0.9591861802229824
El valor de Beta 3 se encuentra entre -1692464.0236537529 y 1692520.795445095
El valor de Beta 4 se encuentra entre -0.008816359228933339 y -0.0014961207978606925
El valor de Beta 5 se encuentra entre -0.43978461876486685 y -0.349811802218163
El valor de Beta 6 se encuentra entre 1.8605932167469759 y 2.3194145517865024
El valor de Beta 7 se encuentra entre 0.2692452697751165 y 0.32342076264039304
El valor de Beta 8 se encuentra entre 0.19426763196578603 y 0.2514187806244933
El valor de Beta 9 se encuentra entre 0.10121113276469902 y 0.15505692970685309
El valor de Beta 10 se encuentra entre -1537283.3470777536 y 1537235.366618443
El valor de Beta 11 se encuentra entre -843999.6396493326 y 843971.4188505389
El valor de Beta 12 se encuentra entre 0.39863725447

In [21]:
# Comparación de resultados contra reporte automatizado
import statsmodels.api as sm

regressor = sm.OLS(y, x).fit()
print(regressor.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.667
Model:                            OLS   Adj. R-squared:                  0.667
Method:                 Least Squares   F-statistic:                     2333.
Date:                Mon, 31 Mar 2025   Prob (F-statistic):               0.00
Time:                        09:57:54   Log-Likelihood:                -14839.
No. Observations:               15129   AIC:                         2.971e+04
Df Residuals:                   15115   BIC:                         2.981e+04
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.1461      0.009    -16.696      0.0

## Calculo de la matriz de correlaciones 

In [23]:
data = pd.DataFrame(x)
data2 = data.iloc[:, 1:]
data2.corr()

,1,2,3,4,5,6,7,8,9,10,11,12,13,14
1,1.000000,0.514100,0.571408,0.032922,0.174930,-0.016160,0.067719,0.027017,0.352756,0.471753,0.297825,-0.013708,0.384261,0.026539
2,0.514100,1.000000,0.752778,0.085667,0.506395,0.054351,0.177997,-0.129141,0.665649,0.684778,0.272992,0.026642,0.565191,0.076886
3,0.571408,0.752778,1.000000,0.175524,0.348570,0.089529,0.273095,-0.060081,0.762060,0.877162,0.423955,0.053754,0.755621,0.174586
4,0.032922,0.085667,0.175524,1.000000,-0.003937,0.027615,0.060493,-0.009827,0.115713,0.185453,0.015023,-0.083162,0.157513,0.719088
5,0.174930,0.506395,0.348570,-0.003937,1.000000,0.013746,0.025277,-0.271358,0.459375,0.517303,-0.251223,0.053110,0.272619,-0.011695
6,-0.016160,0.054351,0.089529,0.027615,0.013746,1.000000,0.387323,0.022265,0.061233,0.057819,0.077022,-0.018708,0.071134,0.033691
7,0.067719,0.177997,0.273095,0.060493,0.025277,0.387323,1.000000,0.050645,0.242958,0.158603,0.268450,-0.000164,0.274204,0.064159
8,0.027017,-0.129141,-0.060081,-0.009827,-0.271358,0.022265,0.050645,1.000000,-0.147678,-0.159999,0.176902,-0.024375,-0.099168,-0.008460
9,0.352756,0.665649,0.762060,0.115713,0.459375,0.061233,0.242958,-0.147678,1.000000,0.754606,0.160582,0.119504,0.708890,0.114707
10,0.471753,0.684778,0.877162,0.185453,0.517303,0.057819,0.158603,-0.159999,0.754606,1.000000,-0.063026,-0.001952,0.730892,0.185735


In [24]:
# Eliminando la fila 2 por el valor de p mas alto
X_nueva = np.delete(x, 2, 1)

regressor = sm.OLS(y, X_nueva).fit()
print(regressor.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.667
Model:                            OLS   Adj. R-squared:                  0.667
Method:                 Least Squares   F-statistic:                     2528.
Date:                Mon, 31 Mar 2025   Prob (F-statistic):               0.00
Time:                        09:57:54   Log-Likelihood:                -14839.
No. Observations:               15129   AIC:                         2.970e+04
Df Residuals:                   15116   BIC:                         2.980e+04
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.1472      0.008    -19.619      0.0

In [25]:
# Eliminando la fila 12 por que tiene el valor de p mas alto
X_nueva = np.delete(x, 12, 1)

regressor = sm.OLS(y, X_nueva).fit()
print(regressor.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.606
Model:                            OLS   Adj. R-squared:                  0.606
Method:                 Least Squares   F-statistic:                     1941.
Date:                Mon, 31 Mar 2025   Prob (F-statistic):               0.00
Time:                        09:57:54   Log-Likelihood:                -16113.
No. Observations:               15129   AIC:                         3.225e+04
Df Residuals:                   15116   BIC:                         3.235e+04
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.2137      0.009    -22.694      0.0

In [26]:
# Eliminando la fila 4 por que tiene el valor de p mas alto
X_nueva = np.delete(x, 4, 1)

regressor = sm.OLS(y, X_nueva).fit()
print(regressor.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.667
Model:                            OLS   Adj. R-squared:                  0.667
Method:                 Least Squares   F-statistic:                     2527.
Date:                Mon, 31 Mar 2025   Prob (F-statistic):               0.00
Time:                        09:57:55   Log-Likelihood:                -14840.
No. Observations:               15129   AIC:                         2.971e+04
Df Residuals:                   15116   BIC:                         2.981e+04
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.1455      0.009    -16.643      0.0